In [66]:
import csv
import uuid
import datetime
from string import Template

In [67]:
inputCSVfile = '../output/predictions.csv'
outputTrigFile = '../output/classifications.trig'
outputGraph = 'http://resource.swissartresearch.net/graph/classifications'
dateTime = datetime.datetime.now()
addTimestampToGraph = True

In [68]:
if addTimestampToGraph:
    outputGraph = outputGraph + '-' + dateTime.strftime("%Y-%m-%d-%H-%M-%S")

In [69]:
data = []
with open(inputCSVfile, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data.append(row)

In [70]:
# Debug
data = data[:10]

In [71]:
namespaces = """
@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.

"""


In [72]:
output = namespaces

In [73]:
typeTemplate = Template("""
    <$graph> {
        <$assignedType> a crm:E52_Type ;
            rdfs:label "$label" .
    }
""")
for prediction in set([d['prediction'] for d in data]):
    typeUri = 'https://resource.swissartresearch.net/type/classification/' + prediction.replace(' ','')
    output += typeTemplate.substitute(graph=outputGraph,
                                     assignedType=typeUri,
                                     label=prediction)

In [74]:
technique = 'https://github.com/swiss-art-research-net/bso-image-classification/commit/583c8891df9739723f81dfc1ac127b06e411ec34'

classificationTemplate = Template("""
    <$graph> {
    
        <$classification> a crm:E13_Attribute_Assignment ;
            crm:P140_assigned_attribute_to <$subject> ;
            crm:P141_assigned <$assignedType> ;
            crm:P4_has_time-span <$classification/date> ;
            crm:P33_used_specific_technique <$technique> ;
            rdf:value "$confidence"^^xsd:float .
            
        <$classification/date> a crm:E52_Time-Span ;
            crm:P82_at_some_time_within "$dateTime"^^xsd:dateTime .
    }
""")

for row in data:
    classificationUri = 'https://resource.swissartresearch.net/classification/' + str(uuid.uuid4())
    subjectUri = 'https://resource.swissartresearch.net/artwork/' + row['id']
    typeUri = 'https://resource.swissartresearch.net/type/classification/' + row['prediction'].replace(' ','')
    output += classificationTemplate.substitute(graph=outputGraph, 
                                  classification=classificationUri, 
                                  subject=subjectUri, 
                                  assignedType=typeUri,
                                  technique=technique,
                                  confidence=row['confidence'],
                                  dateTime=dateTime.strftime("%Y-%m-%dT%H:%M:%S"))

In [75]:
print(output)


@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.


    <http://resource.swissartresearch.net/graph/classifications-2021-03-23-14-42-12> {
        <https://resource.swissartresearch.net/type/classification/landscape> a crm:E52_Type ;
            rdfs:label <landscape> .
    }

    <http://resource.swissartresearch.net/graph/classifications-2021-03-23-14-42-12> {
        <https://resource.swissartresearch.net/type/classification/notlandscape> a crm:E52_Type ;
            rdfs:label <not landscape> .
    }

    <http://resource.swissartresearch.net/graph/classifications-2021-03-23-14-42-12> {
    
        <https://resource.swissartresearch.net/classification/02b8348d-1e32-49ca-baae-285c8155bb7b> a crm:E13_Attribute_Assignment ;
            crm:P140_assigned_attribute_to <https://resource.swissartresearch.net/artwork/zbz-010378524> ;
            crm:P141_assigned <https://resou